In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('gelu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 40s 114ms/step - loss: 2.1028 - accuracy: 0.2594 - val_loss: 1.7500 - val_accuracy: 0.4072
Epoch 2/50
313/313 [==============================] - 34s 108ms/step - loss: 1.5679 - accuracy: 0.4468 - val_loss: 1.5141 - val_accuracy: 0.4840
Epoch 3/50
313/313 [==============================] - 34s 108ms/step - loss: 1.4243 - accuracy: 0.5057 - val_loss: 1.3464 - val_accuracy: 0.5286
Epoch 4/50
313/313 [==============================] - 34s 110ms/step - loss: 1.3264 - accuracy: 0.5454 - val_loss: 1.4345 - val_accuracy: 0.4955
Epoch 5/50
313/313 [==============================] - 33s 107ms/step - loss: 1.2436 - accuracy: 0.5767 - val_loss: 1.5520 - val_accuracy: 0.4715
Epoch 6/50
313/313 [==============================] - 34s 108ms/step - loss: 1.1703 - accuracy: 0.6062 - val_loss: 1.2886 - val_accuracy: 0.5471
Epoch 7/50
313/313 [==============================] - 34s 109ms/step - loss: 1.1158 - accuracy: 0.6225 - val_loss: 1.2532 - val_ac

In [10]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.8701120615005493, 1.533719539642334, 1.4028968811035156, 1.307350754737854, 1.2292102575302124, 1.1605645418167114, 1.104544758796692, 1.0542913675308228, 1.0010123252868652, 0.9584172964096069, 0.912586510181427, 0.8750619888305664, 0.840369462966919, 0.8023239970207214, 0.7691945433616638, 0.7300525903701782, 0.6942577362060547, 0.6634646654129028, 0.6282963156700134, 0.5992242097854614, 0.5625944137573242, 0.5315674543380737, 0.5079725980758667, 0.48014429211616516, 0.44680413603782654, 0.4206153452396393, 0.3892291486263275, 0.3700101673603058, 0.34690916538238525, 0.323626846075058, 0.3040904998779297, 0.2827153205871582, 0.2639458179473877, 0.23792922496795654, 0.21705031394958496, 0.20323660969734192, 0.1924695372581482, 0.18475812673568726, 0.16889382898807526, 0.1556457132101059, 0.1396491527557373, 0.1358380764722824, 0.13096413016319275, 0.1190633773803711, 0.11069712787866592, 0.1020185574889183, 0.09099676460027695, 0.09105625748634338, 0.08355987817049026, 0.0